## Imports

In [ ]:
!pip3 install openai==0.27.0 --quiet

from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata

import openai
openai.api_key = userdata.get("")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 2.5 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
import pandas as pd
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import re
import time
import json
from sklearn.metrics import f1_score, cohen_kappa_score
import numpy as np

In [3]:
MODEL = "gpt-4o-2024-08-06"

model_ids = [model.id for model in openai.Model.list().data]
assert MODEL in model_ids

In [ ]:
# Change FA and ABLATION in between ablations for same FA
FA = "4"
ABLATION = f"_2_ICL"

DATA_PATH = ""
PROMPT_PATH = ""
RESULTS_PATH = ""

ROLE_CONTENT_DELIM = "!~*~!"

# Add this if going from I/O to ICL
LINE_DELIM = "\n!@#@!\n"

SEED = 312
N_BOOTSTRAP = 5000

# Number of few-shot instances * 2 + 1
N_PROMPT_MESSAGES = 5

## Data

In [ ]:
df = pd.read_csv(DATA_PATH)

df.head()

## Prompt

In [ ]:
with open(PROMPT_PATH, 'r', encoding='utf-8') as f:
    prompt = f.read()

prompt

'system!~*~!You are a helpful teacher\'s assistant whose job it is to score middle school students\' short answer formative assessment question responses in the Earth Science domain.\n\nIn this formative assessment, students complete an engineering task that requires them to evaluate two designs of a schoolyard on a 4x4 grid pursuant to several constraints.\n\nStudents are presented with the following information:\n\nA fictitious student, Morgan, has created two different designs. Morgan wants to test both of her designs to see which one is better.\n\nMorgan tested her first design (Design1) with the following inputs:\nRainfall_Inches = 6\nBuilding_Squares = 6\nGrassy_Squares = 4\nPlay_Squares = 3\nParking_Squares = 3\nAccessible_Squares = 8\n\nDesign1 produced the following outputs:\nRunoff_Inches = 5.49\nCost = $732,000\n\nMorgan tested her second design (Design2) with the following inputs:\nRainfall_Inches = 3\nBuilding_Squares = 4\nGrassy_Squares = 3\nPlay_Squares = 4\nParking_Squa

## API Call

In [ ]:
def get_openai_response(messages):
  start_time = time.time()
  response = openai.ChatCompletion.create(
      model=MODEL,
      messages=messages,
      temperature=0,
      response_format={"type": "json_object"},
      seed=SEED)
  total_time = time.time()-start_time
  total_tokens = response["usage"]["total_tokens"]
  generation = response["choices"][0]["message"]["content"].strip()

  return generation, total_time, total_tokens

In [ ]:
response = get_openai_response([{"role":"system","content":"Confirm that the api is working. Respond using the following JSON schema: {'working':bool, 'explanation':str}"}])
print(response)

('{\n    "working": true,\n    "explanation": "The API is functioning correctly as there are no reported issues or errors in the current environment."\n}', 0.6875457763671875, 64)


## Generations

In [ ]:
results = [["llm_score","total_time_s","total_tokens"]]

prompt_messages = prompt.split(LINE_DELIM)
assert len(prompt_messages) == N_PROMPT_MESSAGES

system_role, system_content = prompt_messages[0].split(ROLE_CONTENT_DELIM)
user_role1, user_content1 = prompt_messages[1].split(ROLE_CONTENT_DELIM)
assistant_role1, assistant_content1 = prompt_messages[2].split(ROLE_CONTENT_DELIM)
user_role2, user_content2 = prompt_messages[3].split(ROLE_CONTENT_DELIM)
assistant_role2, assistant_content2 = prompt_messages[4].split(ROLE_CONTENT_DELIM)

for idx,row in df.iterrows():
  messages = [
      {"role":system_role,"content":system_content},
      {"role":user_role1,"content":user_content1},
      {"role":assistant_role1,"content":assistant_content1},
      {"role":user_role2,"content":user_content2},
      {"role":assistant_role2,"content":assistant_content2},
      {"role":"user","content":row['response']}
  ]

  generation, total_time, total_tokens = get_openai_response(messages)
  generation_data = json.loads(generation)

  results.append([generation_data["score"],total_time,total_tokens])

  print(f"FINISHED FA{FA} STUDENT {idx}.")

FINISHED FA4 STUDENT 0.
FINISHED FA4 STUDENT 1.
FINISHED FA4 STUDENT 2.
FINISHED FA4 STUDENT 3.
FINISHED FA4 STUDENT 4.
FINISHED FA4 STUDENT 5.
FINISHED FA4 STUDENT 6.
FINISHED FA4 STUDENT 7.
FINISHED FA4 STUDENT 8.
FINISHED FA4 STUDENT 9.
FINISHED FA4 STUDENT 10.
FINISHED FA4 STUDENT 11.
FINISHED FA4 STUDENT 12.
FINISHED FA4 STUDENT 13.
FINISHED FA4 STUDENT 14.
FINISHED FA4 STUDENT 15.
FINISHED FA4 STUDENT 16.
FINISHED FA4 STUDENT 17.
FINISHED FA4 STUDENT 18.
FINISHED FA4 STUDENT 19.
FINISHED FA4 STUDENT 20.
FINISHED FA4 STUDENT 21.
FINISHED FA4 STUDENT 22.
FINISHED FA4 STUDENT 23.
FINISHED FA4 STUDENT 24.
FINISHED FA4 STUDENT 25.
FINISHED FA4 STUDENT 26.
FINISHED FA4 STUDENT 27.
FINISHED FA4 STUDENT 28.
FINISHED FA4 STUDENT 29.
FINISHED FA4 STUDENT 30.
FINISHED FA4 STUDENT 31.
FINISHED FA4 STUDENT 32.
FINISHED FA4 STUDENT 33.
FINISHED FA4 STUDENT 34.
FINISHED FA4 STUDENT 35.
FINISHED FA4 STUDENT 36.
FINISHED FA4 STUDENT 37.
FINISHED FA4 STUDENT 38.
FINISHED FA4 STUDENT 39.
FINISHED F

In [ ]:
df_results = pd.DataFrame(results[1:],columns=results[0])
df = pd.concat([df,df_results],axis=1)
df.head(25)

## Save

In [ ]:
df.to_csv(path_or_buf=RESULTS_PATH,index=False)

## Metrics

In [5]:
df = pd.read_csv(RESULTS_PATH)

In [6]:
y_true = df["score"].astype(int)
y_pred = df["llm_score"].astype(int)

f1 = f1_score(y_true, y_pred, average="micro")
qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")

print(f"Micro‑F1: {f1:.4f}")
print(f"QWK:      {qwk:.4f}")

Micro‑F1: 0.7800
QWK:      0.8069


## CIs

In [7]:
rng = np.random.default_rng(SEED)

f1_samples   = []
kappa_samples = []

n = len(df)

for _ in range(N_BOOTSTRAP):
    idx = rng.integers(0, n, n)
    y_t = y_true.iloc[idx].values
    y_p = y_pred.iloc[idx].values

    f1_samples.append(
        f1_score(y_t, y_p, average="micro")
    )
    kappa_samples.append(
        cohen_kappa_score(y_t, y_p, weights="quadratic")
    )

ci_f1   = np.percentile(f1_samples, [2.5, 97.5])
ci_kappa = np.percentile(kappa_samples, [2.5, 97.5])

moe_f1   = (ci_f1[1]   - ci_f1[0])   / 2
moe_kappa = (ci_kappa[1] - ci_kappa[0]) / 2

print(f"Micro‑F1: {f1*100:.2f} ± {moe_f1*100:.2f}")
print(f"QWK:      {qwk*100:.2f} ± {moe_kappa*100:.2f}")

Micro‑F1: 78.00 ± 11.02
QWK:      80.69 ± 15.69
